In [ ]:
%cd /workspace/src

import sys
sys.path = sys.path[2:]

import os.path as osp

import pandas as pd

from utils.general import load_json

excel_path = osp.join("/workspace/src/know/MinKAssess", 'probing_data_gpt2-xl_corrected.xlsx')
avg_rating_path = osp.join("/workspace/src/know/MinKAssess", 'avg_raterdict_202408.json')

avg_rating = load_json(avg_rating_path)
excel = pd.read_excel(excel_path)

In [ ]:
excel.columns

In [ ]:
import numpy as np


data = excel[excel['Probing Element'] != 'relation'].groupby(['Fact_id', 'Probing Element'])[['Rel_id', 'Gold Answer', 'Probing Sentence', 'Disambiguation']].first().reset_index()
# data['Fact_id'] = data['Fact_id'].astype(str)
# data = data[np.isin(data['Fact_id'], np.array(list(avg_rating.keys())))]

In [ ]:
len(avg_rating)

In [ ]:
len(data)

In [ ]:
data.head(20)

In [ ]:
data.columns

In [ ]:
import itertools
from kb.core import Relation, TripleQuery, Entity
from kb.wikidata import TempWikidata, WikidataPrepStage

class Skip(Exception):
    pass
wd = TempWikidata('20210104', WikidataPrepStage.ALMOST_RAW)
errors = []
triple2factid = {}
for i, (fact_id, group) in enumerate(data.groupby('Fact_id')):
    try:
        print('Position', i)
        potential_objects, potential_subjects, relation = None, None, None
        for i, (_, probing_element, rel_id, gold_answer, prompt, disambg) in group.iterrows():
            gold_answer = str(gold_answer)
            gold_answer = gold_answer.replace(" ' ", "'").replace('茅', 'é').replace(" , ", ", ").replace(' - ', '-').lstrip()
            # Correct typos
            if gold_answer == 'Académie Goncourt':
                raise Skip
            if gold_answer == 'United States':
                gold_answer = "United States of America"
            if gold_answer == 'The CW':
                gold_answer = 'CW'
            if gold_answer == 'Shakespeare':
                raise Skip
            if gold_answer == 'Ernst Herzfeld':
                raise Skip
            if gold_answer == 'Subliminal':
                raise Skip
            if gold_answer in ('Université Laval', 'Odell', 'Israeli Arab', 'Edward', 
                               'Provo, Utah', 'EA Sports', 'Lucien', 'Washington', 'first-class', 
                               'arrondissement', 'caliph'):
                raise Skip
            if gold_answer == 'Putin':
                gold_answer = "Vladimir Putin"
            if gold_answer == "Mo'ed":
                gold_answer = "Seder Moed"
            if gold_answer == 'Zaire':
                gold_answer = "DR Congo national football team"
            if gold_answer == 'Lieutenant-Governor':
                gold_answer = "Governor of Van Diemen's Land"
            if gold_answer == 'American':
                gold_answer = "United States of America"
            if gold_answer == 'Major League Baseball ( MLB )':
                gold_answer = 'Major League Baseball'
            if gold_answer == 'Crown Vic':
                gold_answer = "Ford Crown Victoria"
            if gold_answer == 'Ford':
                gold_answer = "Ford Motor Company"
            if gold_answer == 'Ahmad Chalabi':
                gold_answer = "Ahmed Chalabi"
            
            print(gold_answer, relation, type(gold_answer))

            if relation is None:
                relation = Relation(rel_id)
            if probing_element == 'object':
                object_gold_answer = gold_answer
                add = []
                if gold_answer == 'Switzerland':
                    add.append('Swiss national football team')
                potential_objects = list(itertools.chain(*wd.find_from_label([gold_answer] + add).values()))
                
                # Manual Entity Disambiguation
                if gold_answer == 'Chicago Black Hawks':
                    potential_objects = [Entity('Q209636')]
                elif gold_answer == "CW":
                    potential_objects = [Entity('Q212252')]
                elif gold_answer == 'papacy':
                    potential_objects = [Entity('Q9592')]
                elif gold_answer == 'Prague':
                    potential_objects = [Entity("Q1085")]
                elif gold_answer == 'Norway':
                    potential_objects = [Entity('Q20')]
            elif probing_element == 'subject':
                subject_gold_answer = gold_answer
                add = []
                if gold_answer == 'Switzerland':
                    add.append('Swiss national football team')
                potential_subjects = list(itertools.chain(*wd.find_from_label([gold_answer] + add).values()))

                # Manual Entity Disambiguation
                if gold_answer == 'Scottish referendum':
                    raise Skip
                elif gold_answer == 'Fire Emblem':
                    potential_subjects = [Entity("Q1768977")]
                elif gold_answer == 'Pope Francis':
                    potential_subjects = [Entity("Q450675")]
                elif gold_answer == "John Dryden":
                    potential_subjects = [Entity('Q213355')]
                elif gold_answer == 'Gmail':
                    potential_subjects = [Entity('Q9334')]
                elif gold_answer == 'Greg Boyer':
                    potential_subjects = [Entity('Q2329528')]
                elif gold_answer == 'Henrique':
                    potential_subjects = [Entity('Q1606369')]
                elif gold_answer == 'Korn':
                    potential_subjects = [Entity('Q80304')]
                elif gold_answer == 'Paul Wild':
                    potential_subjects = [Entity('Q123536')]
                elif gold_answer == 'Ruby programming language':
                    potential_subjects = [Entity('Q161053')]
                
                    
        
        assert potential_objects is not None and potential_subjects is not None
        print(prompt)
        print(potential_subjects, potential_objects, relation)
        print([x.id for x in potential_subjects], [x.id for x in potential_objects])

        if Entity("Q7554518") in potential_subjects and Entity('Q28575') in potential_objects:
            potential_objects = [Entity("Q192299")]


        triples = list(wd.find(TripleQuery(subject=potential_subjects, relation=relation, object=potential_objects)))
        seen = set()
        uniqueidlist = []
        for obj in triples:
            obj2 = obj.to_sro()
            if obj2 not in seen:
                seen.add(obj2)
                uniqueidlist.append(obj)
        triples = uniqueidlist
        print([(tr.subject.id, tr.relation.id, tr.object.id) for tr in triples])
        if len(triples) > 1:
            print('Disambiguation', disambg)
            triples = [tr for tr in triples if tr.subject.id in (disambg, 'Q22') or tr.object.id in (disambg, 'Q22')]
        assert len(triples) == 1

        triple2factid[triples[0]] = fact_id
    except Skip:
        print('SKIPPING : ', prompt)
    except AssertionError:
        errors.append((subject_gold_answer, relation, object_gold_answer))

In [ ]:
triple2humanscore = [
    (tuple(x.id for x in k.to_sro()), avg_rating[str(v)]) for k,v in triple2factid.items()
]

In [ ]:
triple2humanscore

In [ ]:
from utils.general import dump_json
import random
random.seed(42)
random.shuffle(triple2humanscore)

dump_json(osp.join('know/MinKAssess', 'wikidata_triples_to_human_scores.json'), triple2humanscore)